
# Coursera Capstone Project

## The Battle of Neighborhoods - Final Report (Week 1 and 2)


### Libraries Required

In [1]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans



print('Libraries imported.')

Libraries imported.


# Coursera Capstone - REPORT 

### Contents

1.	Introduction Section : 
	
    1.1 Discussion of the "background situation" leading to the problem at hand:
    
    1.2 Problem to be resolved
    
    1.3 Audience for this project.
    
    
2.	Data Section:  

	2.1 Data of Current Situation (current residence place)
    
    2.2 Data required to resolve the problem
    
    2.3 Data sources and data manipulation
    
    
3.	Methodology section : 

	3.1 Process steps and strategy to resolve the problem
    
    3.2 Data Science Methods, machine learing, mapping tools and exploratory data analysis.
    
    
4.	Results Section 

	Discussion of the results and how they help to take a decision.
    


# 1.	Introduction Section : 
### Discussion of the business problem and the audience who would be interested in this project.


### 1.1 Scenario and  Background

I am a current resident of Paris, France. My current apartment offers me many
advantages and assets, and the most important one is the proximity to many
amenities and venues in the area, both for practical daily life matters (shops,
gyms) and for entertainment (cafés, restaurant..).
We will consider a hypothetical scenario where i had been offered a job
opportunity in Manhattan, NY. Although I am excited about it, I am concerned
about the change that this implies, and I am mindful in keeping my same current.
So, I need to find, in Manhattan, a comparable place where I can move in.
Instead of spending many hours reading articles and posts in google, I decided to
use a data analysis approach based on exploration and clustering of both my
current city (Paris) and the city I am planning to move in (NY), with the objective
of f


### 1.2  Problem to be resolved:   

The challenge to resolve is being able to find an apartment unit in Manhattan NY that offers similar characteristics and benefits to my current situation. Therefore, in order to set a basis for comparison, I want to place subject to the following conditions: 
- Top ammenities in the selected neighborhood shall be similar to current residence (See item 2.1)
- Desirable to have venues such as coffee shops, restaurants Asian Thai, wine stores, gym and food shops
- As a reference, I have included a map of venues near current residence in Louis Morard, Paris, France.

### 1.3 Interested Audience
This analysis might be of a good use (or at least inspiring) to each person or entity
considering a similar move. The approach and methodologies used here
are applicable in variety of cases. The use of FourSquare data and mapping
techniques combined with data analysis will help resolve the key questions arisen. 

# 2. Data Section:
### Description of the data and its sources that will be used to solve the problem

### 2.1 Data of Current Situation
I Currently reside in the 14 th district of Paris, I use Foursquare to identify the
venues around the area of residence which are then shown in the Paris map
shown in methodology and execution in section 3. It serves as a reference for
comparison with the desired future location in
Manhattan NY

### 2.2 Data Required to resolve the problem
In order to make a good choice of a similar apartment in Manhattan NY, the following data is required:
- List/Information on neighborhoods form Manhattan with their Geodata(latitude and longitude).
- Venues and ammenities in the Manhattan neighborhoods (e.g. top 10)

### 2.3 sources and manipulation
The list of Manhattan neighborhoods is worked out during LAB exercise during the course. A csv file was created which will be read in order to create a dataframe and its mapping. The csv file 'mh_neigh_data.csv'  has the following below data structure. The file will be directly read to the Jupiter Notebook for convenience and space savings. The clustering of neighborhoods and mapping will be shown however. An algorythm was used to determine the geodata from Nominatim . The actual algorythm coding may be shown in 'markdown' mode becasues it takes time to run. 

mh_neigh_data.tail():

              Borough    Neighborhood           Latitude    Longitude

          35    Manhattan    Turtle Bay         40.752042    -73.967708

          36    Manhattan    Tudor City         40.746917    -73.971219

          37    Manhattan    Stuyvesant Town    40.731000    -73.974052

          38    Manhattan    Flatiron           40.739673    -73.990947

          39    Manhattan    Hudson Yards       40.756658    -74.000111
  

### 2.4 How the data will be used to solve the problem
The data will be used as follows:
- Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups ( as per Course LAB)


### 2.5 Mapping of Data
The following maps were created to facilitate the analysis and the choice of the palace to live.
- Manhattan map of Neighborhoods
- Manhattan map of clustered venues and neighborhoods



## 3. Methodology section:

This section represents the main component of the report where the data is gathered, prepared for analysis. The tools described are used here and the Notebook cells indicates the execution of steps. 

### The analysis and the stragegy:
The strategy is based on mapping the above described data in section 2.0, in order to facilitate the choice of a candidate places for accomodation. The choice is made based on the demands imposed : similar venues to Paris, France. This visual approach and maps with popups labels allow quick identification of  location, thus  making the selection very easy. 

The procesing of these DATA and its mapping will allow to answer the key questions to make a decision:
- What are the venues of the best place to live?
- How venues distribute among Manhattan neighborhoods ?

#      METHODOLOY EXECUTION  - Mapping Data

## Paris Map - Current residence and venues in neighborhood for comparison to future Manhattan place

In [3]:
# Louis Morard, District 14, Paris
address = 'Rue Louis Morard, Paris'
geolocator = Nominatim(user_agent="http")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris home are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris home are 48.8287779, 2.3204412.


In [4]:
neighborhood_latitude=48.8287779
neighborhood_longitude=2.3204412

## Dial FourSquare to find venues around current residence in Paris

In [5]:
# @hidden_cell
CLIENT_ID = 'HFB3WUZM20PVRXFLDVZZU3SSLLMAE2Y5EJRHRCYUEBVXOTJI' # your Foursquare ID
CLIENT_SECRET = 'LHLAWSRPKVNMPJDEO0MLQF4E205V4I1VTQB3ZVAMTYNC4MWE' # your Foursquare Secret
VERSION = '20200716' # Foursquare API version

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [6]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=HFB3WUZM20PVRXFLDVZZU3SSLLMAE2Y5EJRHRCYUEBVXOTJI&client_secret=LHLAWSRPKVNMPJDEO0MLQF4E205V4I1VTQB3ZVAMTYNC4MWE&v=20200716&ll=48.8287779,2.3204412&radius=500&limit=100'

In [7]:
# results display is hidden for report simplification 
results = requests.get(url).json()
#results 


#### function that extracts the category of the venue - borrow from the Foursquare lab.

In [8]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
venues = results['response']['groups'][0]['items']
Parisnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Parisnearby_venues =Parisnearby_venues.loc[:, filtered_columns]
# filter the category for each row
Parisnearby_venues['venue.categories'] = Parisnearby_venues.apply(get_category_type, axis=1)
# clean columns
Parisnearby_venues.columns = [col.split(".")[-1] for col in Parisnearby_venues.columns]

Parisnearby_venues.shape

C:\Users\karimmouqadem\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


(28, 4)

In [10]:
# Venues near current Paris residence place
Parisnearby_venues.head(10)

,name,categories,lat,lng
0,Les Petits Plats,French Restaurant,48.828652,2.322138
1,Le Jeroboam,Modern European Restaurant,48.829527,2.317492
2,Chez Kim Ly,Vietnamese Restaurant,48.828537,2.316201
3,La Régalade,French Restaurant,48.825026,2.320244
4,Le 14 Juillet,French Restaurant,48.828450,2.316137
5,Au Moulin Vert,French Restaurant,48.829773,2.323024
6,Le Petit Baigneur,French Restaurant,48.831712,2.324259
7,Le Severo,French Restaurant,48.832074,2.324704
8,Citadines Didot Montparnasse,Hotel,48.827892,2.315296
9,Place Victor et Hélène Basch,Plaza,48.828033,2.326719


## Map of Paris - 14 ème arrondissement residence place with venues in Neighborhood - for reference

In [20]:
latitude=48.8287779
longitude=2.3204412
# create map of Paris 14 ème place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=18)
# add markers to map
for lat, lng, label in zip(Parisnearby_venues['lat'], Parisnearby_venues['lng'], Parisnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=7,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_sg)  
    
map_sg

# MANHATTAN NEIGHBORHOODS - DATA AND MAPPING 
## Cluster neighborhood data was produced with Foursquare during course lab work. A csv file was produced containing the neighborhoods around the 40 Boroughs. Now, the csv file is just read for convenience and consolidation of report.

In [12]:
# Read csv file with clustered neighborhoods with geodata
manhattan_data  = pd.read_csv('mh_neigh_data.csv') 
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,Manhattan,Marble Hill,40.876551,-73.910660,2
1,Manhattan,Chinatown,40.715618,-73.994279,2
2,Manhattan,Washington Heights,40.851903,-73.936900,4
3,Manhattan,Inwood,40.867684,-73.921210,3
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0


In [12]:
manhattan_data.tail()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
35,Manhattan,Turtle Bay,40.752042,-73.967708,3
36,Manhattan,Tudor City,40.746917,-73.971219,3
37,Manhattan,Stuyvesant Town,40.731000,-73.974052,4
38,Manhattan,Flatiron,40.739673,-73.990947,3
39,Manhattan,Hudson Yards,40.756658,-74.000111,2


# Manhattan Borough neighborhoods - data with top 10 clustered venues


In [17]:
manhattan_merged = pd.read_csv('manhattan_merged.csv')
manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Coffee Shop,Discount Store,Yoga Studio,Steakhouse,Supplement Shop,Tennis Stadium,Shoe Store,Gym,Bank,Seafood Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,2,Chinese Restaurant,Cocktail Bar,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Salon / Barbershop,Noodle House,Bakery,Bubble Tea Shop,Ice Cream Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Café,Bakery,Mobile Phone Shop,Pizza Place,Sandwich Place,Park,Gym,Latin American Restaurant,Tapas Restaurant,Mexican Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,3,Mexican Restaurant,Lounge,Pizza Place,Café,Wine Bar,Bakery,American Restaurant,Park,Frozen Yogurt Shop,Spanish Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Mexican Restaurant,Coffee Shop,Café,Deli / Bodega,Pizza Place,Liquor Store,Indian Restaurant,Sushi Restaurant,Sandwich Place,Yoga Studio


# Map of Manhattan neighborhoods with top 10 clustered venues

## popups  allow us to identify each neighborhood and the cluster of venues around it in order to proceed to examine in more detail in the next cell

In [18]:
# create map of Manhattan using latitude and longitude values from Nominatim
latitude= 40.7308619
longitude= -73.9871558 

kclusters=5
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=20,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
  # add markers for rental places to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)    
    
       
map_clusters

## Examine a paticular Cluster - print venues

### After examining several cluster data , I concluded that cluster # 3 resembles closer to the Paris place, therefore providing guidance as to where to look for the future apartment .

### Assigning a value to 'kk' to explore a given cluster.

In [15]:
## kk is the cluster number to explore
kk = 3
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == kk, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,Mexican Restaurant,Lounge,Pizza Place,Café,Wine Bar,Bakery,American Restaurant,Park,Frozen Yogurt Shop,Spanish Restaurant
5,Manhattanville,Deli / Bodega,Italian Restaurant,Seafood Restaurant,Mexican Restaurant,Sushi Restaurant,Beer Garden,Coffee Shop,Falafel Restaurant,Bike Trail,Other Nightlife
10,Lenox Hill,Sushi Restaurant,Italian Restaurant,Coffee Shop,Gym / Fitness Center,Pizza Place,Burger Joint,Deli / Bodega,Gym,Sporting Goods Shop,Thai Restaurant
12,Upper West Side,Italian Restaurant,Bar,Bakery,Vegetarian / Vegan Restaurant,Indian Restaurant,Coffee Shop,Cosmetics Shop,Wine Bar,Mexican Restaurant,Sushi Restaurant
16,Murray Hill,Sandwich Place,Hotel,Japanese Restaurant,Gym / Fitness Center,Coffee Shop,Salon / Barbershop,Burger Joint,French Restaurant,Bar,Italian Restaurant
17,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bakery,Nightclub,Theater,Art Gallery,Seafood Restaurant,American Restaurant,Hotel
18,Greenwich Village,Italian Restaurant,Sushi Restaurant,French Restaurant,Clothing Store,Chinese Restaurant,Café,Indian Restaurant,Bakery,Seafood Restaurant,Electronics Store
27,Gramercy,Italian Restaurant,Restaurant,Thrift / Vintage Store,Cocktail Bar,Bagel Shop,Coffee Shop,Pizza Place,Mexican Restaurant,Grocery Store,Wine Shop
29,Financial District,Coffee Shop,Hotel,Gym,Wine Shop,Steakhouse,Bar,Italian Restaurant,Pizza Place,Park,Gym / Fitness Center
31,Noho,Italian Restaurant,French Restaurant,Cocktail Bar,Gift Shop,Bookstore,Grocery Store,Mexican Restaurant,Hotel,Sushi Restaurant,Coffee Shop


---

# 4 Results

## ONE CONSOLIDATED MAP
### Let's consolidate all the required inforamtion to make the apartment selection in one map


# Map of Manhattan with  cluster of venues
###  Bubbles are the clusters of venues

In [19]:
# create map of Manhattan using latitude and longitude values from Nominatim
latitude= 40.7308619
longitude= -73.9871558

map_mh_one = folium.Map(location=[latitude, longitude], zoom_start=13.3)


# set color scheme for the clusters
kclusters=5
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=15,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_mh_one)

    # Adds tool to the top right
from folium.plugins import MeasureControl
map_mh_one.add_child(MeasureControl())

# Measurement ruler icon tool to measure distances in map
from folium.plugins import FloatImage
url = ('https://media.licdn.com/mpr/mpr/shrinknp_100_100/AAEAAQAAAAAAAAlgAAAAJGE3OTA4YTdlLTkzZjUtNDFjYy1iZThlLWQ5OTNkYzlhNzM4OQ.jpg')
FloatImage(url, bottom=5, left=85).add_to(map_mh_one)

map_mh_one

# Problem Resolution - Select the similar apartment

## The above consolidate map was used to explore options.  
### After examining, I have chosen the locations that meets the requirements which will assess to make a choice.

- Financial District Neighborhood, Cluster 3: 
    Financial District having Gym, Hotels and Restuarents similar to Paris residence.
    

## Similar Venue - Financil District in Cluster 3

In [17]:
## kk is the cluster number to explore
kk = 3
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == kk, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,Mexican Restaurant,Lounge,Pizza Place,Café,Wine Bar,Bakery,American Restaurant,Park,Frozen Yogurt Shop,Spanish Restaurant
5,Manhattanville,Deli / Bodega,Italian Restaurant,Seafood Restaurant,Mexican Restaurant,Sushi Restaurant,Beer Garden,Coffee Shop,Falafel Restaurant,Bike Trail,Other Nightlife
10,Lenox Hill,Sushi Restaurant,Italian Restaurant,Coffee Shop,Gym / Fitness Center,Pizza Place,Burger Joint,Deli / Bodega,Gym,Sporting Goods Shop,Thai Restaurant
12,Upper West Side,Italian Restaurant,Bar,Bakery,Vegetarian / Vegan Restaurant,Indian Restaurant,Coffee Shop,Cosmetics Shop,Wine Bar,Mexican Restaurant,Sushi Restaurant
16,Murray Hill,Sandwich Place,Hotel,Japanese Restaurant,Gym / Fitness Center,Coffee Shop,Salon / Barbershop,Burger Joint,French Restaurant,Bar,Italian Restaurant
17,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bakery,Nightclub,Theater,Art Gallery,Seafood Restaurant,American Restaurant,Hotel
18,Greenwich Village,Italian Restaurant,Sushi Restaurant,French Restaurant,Clothing Store,Chinese Restaurant,Café,Indian Restaurant,Bakery,Seafood Restaurant,Electronics Store
27,Gramercy,Italian Restaurant,Restaurant,Thrift / Vintage Store,Cocktail Bar,Bagel Shop,Coffee Shop,Pizza Place,Mexican Restaurant,Grocery Store,Wine Shop
29,Financial District,Coffee Shop,Hotel,Gym,Wine Shop,Steakhouse,Bar,Italian Restaurant,Pizza Place,Park,Gym / Fitness Center
31,Noho,Italian Restaurant,French Restaurant,Cocktail Bar,Gift Shop,Bookstore,Grocery Store,Mexican Restaurant,Hotel,Sushi Restaurant,Coffee Shop


# Venue Selection

## Using the "one map" above, I was able to explore all possibilities since the popups provide the information needed for a good decision.

### Financial District having Gyms, Hotels and Restuarents similar toParis residence is my preferabe choice for a future residence.

## Based on current Paris venue, I feel that Cluster 3 type of venues is a closer resemblance to my current place. That means that Financial District is a better choice since it has Hotels and Restuarent and is worth the conveniences it provides.

---